In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data = pd.read_csv('./movie_data/tmdb_5000_movies.csv')

FileNotFoundError: [Errno 2] No such file or directory: './movie_data/tmdb_5000_movies.csv'

In [ ]:
data.head(2)

In [11]:
data.shape

(4803, 20)

In [12]:
data = data[['id','genres', 'vote_average', 'vote_count','popularity','title',  'keywords', 'overview']]

In [13]:

tmp_m = data['vote_count'].quantile(0.89)
tmp_m

1683.8999999999987

In [14]:
tmp_data = data.copy().loc[data['vote_count'] >= tmp_m]
tmp_data.shape

(529, 8)

In [15]:
del tmp_data

m = data['vote_count'].quantile(0.9)
data = data.loc[data['vote_count'] >= m]

In [16]:

data.head()

,id,genres,vote_average,vote_count,popularity,title,keywords,overview
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [17]:
C = data['vote_average'].mean()

In [19]:
print(C)
print(m)

6.9629937629937615
1838.4000000000015


In [20]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    
    return ( v / (v+m) * R ) + (m / (m + v) * C)

In [21]:
data['score'] = data.apply(weighted_rating, axis = 1)

In [22]:
data['score'] = data.apply(weighted_rating, axis = 1)

In [23]:
data.head(5)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",6.918271
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,6.493333
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,7.492998
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",6.500396


In [24]:
data.shape

(481, 9)

In [25]:
data[['genres', 'keywords']].head(2)

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."


In [26]:
data['genres'] = data['genres'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval)

In [27]:
data[['genres', 'keywords']].head(2)

,genres,keywords
0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 1463, 'name': 'culture clash'}, {'id':..."
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na..."


In [28]:
data['genres'] = data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))
data['keywords'] = data['keywords'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : " ".join(x))

In [29]:
data.head(2)

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
0,19995,Action Adventure Fantasy Science Fiction,7.2,11800,150.437577,Avatar,culture clash future space war space colony so...,"In the 22nd century, a paraplegic Marine is di...",7.168053
1,285,Adventure Fantasy Action,6.9,4500,139.082615,Pirates of the Caribbean: At World's End,ocean drug abuse exotic island east india trad...,"Captain Barbossa, long believed to be dead, ha...",6.918271


In [30]:
data.to_csv('./movie_data/pre_tmdb_5000_movies.csv', index = False)

In [31]:

data.genres.head(2)

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
Name: genres, dtype: object

In [32]:
count_vector = CountVectorizer(ngram_range=(1, 3))

In [33]:
c_vector_genres = count_vector.fit_transform(data['genres'])

In [34]:
c_vector_genres.shape

(481, 364)

In [35]:
gerne_c_sim = cosine_similarity(c_vector_genres, c_vector_genres).argsort()[:, ::-1]

In [36]:
gerne_c_sim.shape

(481, 481)

In [37]:
def get_recommend_movie_list(df, movie_title, top=30):
    # 특정 영화와 비슷한 영화를 추천해야 하기 때문에 '특정 영화' 정보를 뽑아낸다.
    target_movie_index = df[df['title'] == movie_title].index.values
    
    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = gerne_c_sim[target_movie_index, :top].reshape(-1)
    #본인을 제외
    sim_index = sim_index[sim_index != target_movie_index]

    #data frame으로 만들고 vote_count으로 정렬한 뒤 return
    result = df.iloc[sim_index].sort_values('score', ascending=False)[:10]
    return result

In [38]:
get_recommend_movie_list(data, movie_title='The Dark Knight Rises')

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
65,155,Drama Action Crime Thriller,8.2,12002,187.322927,The Dark Knight,dc comics crime fighter secret identity scarec...,Batman raises the stakes in his war on crime. ...,8.035690
2091,274,Crime Drama Thriller,8.1,4443,18.174804,The Silence of the Lambs,based on novel psychopath horror suspense seri...,"FBI trainee, Clarice Starling ventures into a ...",7.767228
2760,264644,Drama Thriller,8.1,2757,66.113340,Room,based on novel carpet isolation kidnapping imp...,Jack is a young boy of 5 years old who has liv...,7.645138
351,1422,Drama Thriller Crime,7.9,4339,63.429157,The Departed,undercover boston police friends mafia underco...,"To take down South Boston's Irish Mafia, the p...",7.621146
1850,111,Action Crime Drama Thriller,8.0,2948,70.105981,Scarface,miami corruption capitalism cuba prohibition b...,After getting a green card in exchange for ass...,7.601698
4337,103,Crime Drama,8.0,2535,58.845025,Taxi Driver,vietnam veteran taxi obsession drug dealer nig...,A mentally unstable Vietnam War veteran works ...,7.564085
1051,146233,Drama Thriller Crime,7.9,3085,88.496873,Prisoners,pennsylvania kidnapping maze vigilante rural s...,When Keller Dover's daughter and her friend go...,7.550121
828,24,Action Crime,7.7,4949,79.754966,Kill Bill: Vol. 1,japan coma martial arts kung fu underworld yak...,An assassin is shot at the altar by her ruthle...,7.500378
3701,641,Crime Drama,7.9,2443,11.573034,Requiem for a Dream,drug addiction junkie heroin speed diet unsoci...,The hopes and dreams of four ambitious people ...,7.497657
1829,6977,Crime Drama Thriller,7.7,3003,53.645267,No Country for Old Men,texas drug traffic hitman united states–mexico...,"Llewelyn Moss stumbles upon dead bodies, $2 mi...",7.420140


In [39]:
data[data['title'] == 'The Dark Knight Rises']

,id,genres,vote_average,vote_count,popularity,title,keywords,overview,score
3,49026,Action Crime Drama Thriller,7.6,9106,112.31295,The Dark Knight Rises,dc comics crime fighter terrorist secret ident...,Following the death of District Attorney Harve...,7.492998
